In [16]:
#make circos plot with each TF_added and/or Promoter around the edge of the circle, with links from TFs to their target Promoters
#the links are colored in a spectrum from blue to red based on the strength of the interaction (fold_change)


In [17]:
library(circlize)
library(grid)
library(gridBase)
library(ComplexHeatmap)
library(ggplot2)

In [18]:
#read in data
df <- read.table("../../../data/luminescence/coexpression_for_manuscript/coexpression_data_for_circos_plot_fold_change_significant.tsv", header = TRUE, sep = "\t")
# df <- read.table("../../../data/luminescence/coexpression_for_manuscript/coexpression_data_for_circos_plot_fold_change_significant.csv", header = TRUE, sep = ",")
# df <- read.table("../../../data/luminescence/coexpression_for_manuscript/coexpression_data_for_circos_plot_fold_change.tsv", header = TRUE, sep = "\t")

In [19]:
#keep only the Promoter	TF_added and fold_change columns
df <- df[,c(1,2,8,9,10)]

In [20]:
#swap the first two columns
df <- df[, c(2,1,3:ncol(df))]

In [21]:
df

TF_added,Promoter,fold_change,log2_fc,significance
<chr>,<chr>,<dbl>,<dbl>,<chr>
ANAC032,ANAC032,1.3261432,0.40723653,
ANR1,ANAC032,0.4829043,-1.05019069,
ARF18,ANAC032,0.6668839,-0.58449251,
ARF18+ARF9,ANAC032,0.8266927,-0.27457692,
ARF18+ARF9_1μM_NAA,ANAC032,0.8370029,-0.25669554,
ARF18+IAA33,ANAC032,0.6489800,-0.62375415,
ARF18+IAA33_1μM_NAA,ANAC032,0.6730788,-0.57115278,
ARF18_1μM_NAA,ANAC032,0.6828974,-0.55025919,
ARF9,ANAC032,0.7435662,-0.42746698,


In [22]:
#filter out the rows where the signficant column is no
df <- df[df$significance != "no",]

In [23]:
#rename values in TF_added column - rename ARF18+ARF9 to ARF9/18, ARF18+ARF9_1μM_NAA to ARF9/18+NAA, ARF18+IAA33_1μM_NAA to ARF18+IAA33+NAA, ARF9_1μM_NAA to ARF9+NAA, ARF18_1μM_NAA to ARF18+NAA, ARF9+IAA33__1μM_NAA to ARF9+IAA33+NAA, ARF18+ARF9+IAA33_1μM_NAA to ARF9/18+IAA33+NAA, ARF9/18+IAA33_1μM_NAA to ARF9/18+IAA33+NAA, ARF9/18_1μM_NAA to ARF9/18+NAA
df$TF_added <- gsub("ARF18\\+ARF9", "ARF9/18", df$TF_added)
df$TF_added <- gsub("ARF18\\+ARF9_1μM_NAA", "ARF9/18+NAA", df$TF_added)
df$TF_added <- gsub("ARF18\\+IAA33_1μM_NAA", "ARF18+IAA33+NAA", df$TF_added)
df$TF_added <- gsub("ARF9_1μM_NAA", "ARF9+NAA", df$TF_added)
df$TF_added <- gsub("ARF18_1μM_NAA", "ARF18+NAA", df$TF_added)
df$TF_added <- gsub("ARF9\\+IAA33__1μM_NAA", "ARF9+IAA33+NAA", df$TF_added)
df$TF_added <- gsub("ARF18\\+ARF9\\+IAA33_1μM_NAA", "ARF9/18+IAA33+NAA", df$TF_added)
df$TF_added <- gsub("ARF9/18\\+IAA33_1μM_NAA", "ARF9/18+IAA33+NAA", df$TF_added)
df$TF_added <- gsub("ARF9/18_1μM_NAA", "ARF9/18+NAA", df$TF_added)


In [24]:
# #show unique values in TF_added column
# unique(df$TF_added)

In [25]:
df$log2_fc

[1]  0.40723653 -1.05019069 -0.58449251 -0.27457692 -0.25669554 -0.62375415
 [7] -0.57115278 -0.55025919 -0.42746698 -0.34441462 -0.24443803 -1.05029360
[13] -2.26602952 -2.03111510 -1.82781011 -2.35509360 -2.68620880 -2.16769327
[19] -1.88814915 -1.70250140 -1.57522935 -1.49193672  3.86936815 -0.08770882
[25] -1.55359560 -1.57051230 -1.40259553 -1.46900954 -1.37041380 -1.43852643
[31]  0.67997010  0.87390061  2.82088502  0.35798031 -0.41296608 -0.33795848
[37] -0.45547349 -0.23467278 -0.26533743 -0.22550615 -0.34803646  0.25476311
[43]  1.33050629  0.65614249  0.91153728  0.67292085  0.89144733 -1.05193204
[49] -1.08503153 -0.84449385 -0.91581173 -0.75754782 -1.04139579 -0.11837383
[55] -0.19805282  2.70340181  3.28704189  0.95644249  0.33776328 -0.27632142
[61] -0.55364318 -0.54714373 -0.53425262 -0.20628840 -0.58359649 -0.41978140
[67]  0.25560825  1.58119843 -0.90520038  0.32795966  0.26163972  0.06226987
[73] -0.17087368  0.33467414 -0.29864800 -0.23895910  0.58196226  0.24652346
[79]  0.43041609  0.40604186

In [26]:
#get highest and lowest fold change values
max_fc <- max(df$log2_fc)
min_fc <- min(df$log2_fc)

col_fun = colorRamp2(c(-4, 0, 4), c("darkblue", "white", "darkorange"))
#select color based on fold_change
df$color <- col_fun(df$log2_fc)


In [27]:
max_fc

[1] 3.869368

In [28]:
min_fc

[1] -2.686209

In [29]:
# help(chordDiagram)

In [30]:
# # circos.par(gap.after = c(rep(5, length(unique(df[[1]]))-1), 15, 
# #                          rep(5, length(unique(df[[2]]))-1), 15))
# #make chord diagram from TF_added to Promoter
# chordDiagram(df[1:2],col=df$color, directional=1,direction.type = c("arrows+diffHeight"), diffHeight=0.04,)
# #add direction of transcription from TF_added to Promoter







In [31]:
# circlize_plot = function() {
#     set.seed(12345)
#     sectors = letters[1:10]
#     circos.initialize(sectors, xlim = c(0, 1))
#     circos.track(ylim = c(0, 1), panel.fun = function(x, y) {
#         circos.points(runif(20), runif(20), cex = 0.5, pch = 16, col = 2)
#         circos.points(runif(20), runif(20), cex = 0.5, pch = 16, col = 3)
#     })
#     circos.track(ylim = c(0, 1), panel.fun = function(x, y) {
#         circos.lines(sort(runif(20)), runif(20), col = 4)
#         circos.lines(sort(runif(20)), runif(20), col = 5)
#     })

#     for(i in 1:10) {
#         circos.link(sample(sectors, 1), sort(runif(10))[1:2], 
#                     sample(sectors, 1), sort(runif(10))[1:2],
#                     col = add_transparency(col_fun(rnorm(1))))
#     }
#     circos.clear()
# }

In [32]:
circos.clear()
pdf(file = "../../../data/luminescence/coexpression_for_manuscript/circos_plot.pdf")
#set circle size 

# plot.new()
# circle_size = unit(1, "snpc") # snpc unit gives you a square region
#set circle margin
circos.par(circle.margin = 0.5) # margin is 0.1 the radius of the circle
#make labels vertical
chordDiagram(df[1:2],col=df$color, directional=1,direction.type = c("arrows+diffHeight"), link.arr.type = "big.arrow", diffHeight = -mm_h(2),annotationTrack = "grid", preAllocateTracks = 1,) #direction.type = c("arrows+diffHeight")
circos.trackPlotRegion(track.index = 1, panel.fun = function(x, y) {
  xlim = get.cell.meta.data("xlim")
  ylim = get.cell.meta.data("ylim")
  sector.name = get.cell.meta.data("sector.index")
  circos.text(mean(xlim), ylim[1] + .1, sector.name, facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.5))
  
}, bg.border = NA)


lgd_links = Legend(at = c(-4, 0, 4), col_fun = col_fun, 
    title_position = "topleft", title = "Log2FC", direction = "horizontal")

lgd_list_horizontal = packLegend(lgd_links, 
    direction = "horizontal")
# draw(lgd_list_horizontal, y = unit(1, "npc") - circle_size, just = "top")
draw(lgd_list_horizontal, x = unit(6, "mm"), y = unit(5, "mm"), just = c("left", "bottom"))
# #save plot as svg
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot.svg", width = 10, height = 10)
# # save plot as pdf
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot.pdf", width = 10, height = 10)
dev.off()
# circos.clear()

png 
  2

In [33]:
#change outside segment colours to match those in previous figures in thesis, remove non-significant edges.
# size of each segment shows how many edges to and from
# if there are tf-promoter interactions i am most interested in then order them to be opposites at 0-180 degrees or 90-270 or o-90, just not next to each other 
#make all labels fit

In [34]:
pdf(file = "../../../data/luminescence/coexpression_for_manuscript/circos_plot_activator.pdf") 
#show only activation
chordDiagram(df[1:2],col=df$color, link.visible = df$log2_fc > 0, directional=1,direction.type = c("arrows+diffHeight"), link.arr.type = "big.arrow", diffHeight = -mm_h(2),annotationTrack = "grid", preAllocateTracks = 1,) #direction.type = c("arrows+diffHeight")
circos.trackPlotRegion(track.index = 1, panel.fun = function(x, y) {
  xlim = get.cell.meta.data("xlim")
  ylim = get.cell.meta.data("ylim")
  sector.name = get.cell.meta.data("sector.index")
  circos.text(mean(xlim), ylim[1] + .1, sector.name, facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.5))
  
}, bg.border = NA)


lgd_links = Legend(at = c(-4, 0, 4), col_fun = col_fun, 
    title_position = "topleft", title = "Log2FC", direction = "horizontal")

lgd_list_horizontal = packLegend(lgd_links, 
    direction = "horizontal")
# draw(lgd_list_horizontal, y = unit(1, "npc") - circle_size, just = "top")
draw(lgd_list_horizontal, x = unit(6, "mm"), y = unit(5, "mm"), just = c("left", "bottom"))
# #save plot as svg
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot_activator.svg", width = 10, height = 10)
# # save plot as pdf
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot_activator.pdf", width = 10, height = 10)
dev.off()
# circos.clear()

png 
  2

In [35]:
pdf(file = "../../../data/luminescence/coexpression_for_manuscript/circos_plot_repressor.pdf") 
#show only repression
chordDiagram(df[1:2],col=df$color, link.visible = df$log2_fc < 0, directional=1,direction.type = c("arrows+diffHeight"), link.arr.type = "big.arrow", diffHeight = -mm_h(2),annotationTrack = "grid", preAllocateTracks = 1,) #direction.type = c("arrows+diffHeight")
circos.trackPlotRegion(track.index = 1, panel.fun = function(x, y) {
  xlim = get.cell.meta.data("xlim")
  ylim = get.cell.meta.data("ylim")
  sector.name = get.cell.meta.data("sector.index")
  circos.text(mean(xlim), ylim[1] + .1, sector.name, facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.5))
  
}, bg.border = NA)


lgd_links = Legend(at = c(-4, 0, 4), col_fun = col_fun, 
    title_position = "topleft", title = "Log2FC", direction = "horizontal")

lgd_list_horizontal = packLegend(lgd_links, 
    direction = "horizontal")
# draw(lgd_list_horizontal, y = unit(1, "npc") - circle_size, just = "top")
draw(lgd_list_horizontal, x = unit(6, "mm"), y = unit(5, "mm"), just = c("left", "bottom"))
#save plot as svg
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot_repressor.svg", width = 10, height = 10)
# # save plot as pdf
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot_repressor.pdf", width = 10, height = 10)

dev.off()
# circos.clear()

png 
  2

In [36]:
#now filter df to exclude values in TF_added column: exclude any values with part of the string containing NAA or IAA3


df = df[!grepl("NAA", df$TF_added),]
df = df[!grepl("IAA3", df$TF_added),]


In [37]:
#show unique TF_added values
unique(df$TF_added)

[1] "ANAC032" "ANR1"    "ARF18"   "ARF9/18" "ARF9"    "NLP6"    "NLP7"   
[8] "DREB26"

In [38]:
#make the plot again without excluded values
circos.clear()
pdf(file = "../../../data/luminescence/coexpression_for_manuscript/circos_plot_no_NAA.pdf")
#set circle size 

# plot.new()
# circle_size = unit(1, "snpc") # snpc unit gives you a square region
#set circle margin
circos.par(circle.margin = 0.5) # margin is 0.1 the radius of the circle
#make labels vertical
chordDiagram(df[1:2],col=df$color, directional=1,direction.type = c("arrows+diffHeight"), link.arr.type = "big.arrow", diffHeight = -mm_h(2),annotationTrack = "grid", preAllocateTracks = 1,) #direction.type = c("arrows+diffHeight")
circos.trackPlotRegion(track.index = 1, panel.fun = function(x, y) {
  xlim = get.cell.meta.data("xlim")
  ylim = get.cell.meta.data("ylim")
  sector.name = get.cell.meta.data("sector.index")
  circos.text(mean(xlim), ylim[1] + .1, sector.name, facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.5))
  
}, bg.border = NA)


lgd_links = Legend(at = c(-4, 0, 4), col_fun = col_fun, 
    title_position = "topleft", title = "Log2FC", direction = "horizontal")

lgd_list_horizontal = packLegend(lgd_links, 
    direction = "horizontal")
# draw(lgd_list_horizontal, y = unit(1, "npc") - circle_size, just = "top")
draw(lgd_list_horizontal, x = unit(6, "mm"), y = unit(5, "mm"), just = c("left", "bottom"))
# #save plot as svg
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot.svg", width = 10, height = 10)
# # save plot as pdf
# ggsave("../../../data/luminescence/coexpression_for_manuscript/circos_plot.pdf", width = 10, height = 10)
dev.off()
# circos.clear()

png 
  2